Notebook for demonstrating the resource estimation pipeline.

In [1]:
from qsharp.estimator import LogicalCounts
from qualtran import Bloq, QUInt
from qualtran.bloqs.mod_arithmetic import CModMulK

**Step 1.** Define the key size $n$.

In [2]:
key_size = 2048

**Step 2.** Create the Bloq which defines the subcircuit dominating the resource cost of the algorithm. In this case (and most cases) it is the controlled modular multiplier.

- `modulus` is the integer to be factored. Since the resource cost is (usually) the same for any $n$-bit integer, for simplicity we take it to be $N = 2^n - 1$.
- `multiplicand` is the classically pre-computed constant which is multiplied by the value in the quantum register, i.e., the number $a$ in the quantum computation $\ket{x} \mapsto \ket{ax \mod N}$.<br>The resource cost of the controlled modular multiplier is also (usually) invariant in the value of $a$. Since $a$ must be coprime to $N$, for simplicity we take it to be $a = N - 1 = 2^n - 2$.

In [3]:
modulus = 2**key_size - 1
multiplicand = 2**key_size - 2

mod_mul = CModMulK(dtype=QUInt(key_size), k=multiplicand, mod=modulus)

**Step 3.** Compute the number of ancilla qubits needed for the subcircuit. To match the implementation of [Gidney and Ekerå](https://doi.org/10.22331/q-2021-04-15-433), we take this to be $2 n$.

In [4]:
ancillas = int(2*key_size)

**Step 4.** Compute the number of times the subcircuit is repeated in the entire algorithm. To again match the implementation of [Gidney and Ekerå](https://doi.org/10.22331/q-2021-04-15-433), we take this to be $1.5 n$.

In [5]:
subcircuit_reps = int(1.5*key_size)

**Step 5.** Convert logical resource estimates from the circuit defined by Qualtran Bloqs into a format that can be input to Microsoft's Azure Quantum Resource Estimator.

In [6]:
def get_bloq_logical_counts(bloq: Bloq, ancillas: int = 0, subcircuit_reps: int = 1) -> LogicalCounts:
    """Perform resource estimation on multiple copies of a Qualtran Bloq.

    Parameters
    ----------
    bloq : Bloq
        Subcircuit which dominates the resource cost
    ancillas : int, optional
        Number of ancillas needed by the Qualtran Bloq, by default 0
    subcircuit_reps : int, optional
        Number of sequential repetitions of the Qualtran Bloq, by default 1

    Returns
    -------
    LogicalCounts
        Resource estimates returned by qsharp.
    """
    # This way of calculating the number of qubits will be inaccurate if a sub_bloq
    # allocates new qubits.

    num_qubits = bloq.signature.n_qubits() + ancillas

    print(f"Logical qubits: {num_qubits}")
    complexity = bloq.t_complexity()
    print(complexity)

    return LogicalCounts(
        {
            "numQubits": num_qubits,
            "tCount": complexity.t * subcircuit_reps,
            "rotationCount": complexity.rotations * subcircuit_reps,
            "rotationDepth": complexity.rotations * subcircuit_reps,
        }
    )

In [7]:
logical_counts = get_bloq_logical_counts(mod_mul, ancillas, subcircuit_reps)

Logical qubits: 6145
T-count:   1.67705e+08
Rotations: 0
Cliffords: 8.38431e+08



**Step 6.** Use Microsoft's Azure Quantum Resource Estimator to obtain physical resources estimates from the logical resource estimates.

In [8]:
estimates = logical_counts.estimate(
    [
        {"qubitParams": {"name": "qubit_gate_ns_e3"}},
        # {"qubitParams": {"name": "qubit_gate_ns_e4"}},
        # {"qubitParams": {"name": "qubit_maj_ns_e4"}, "surfaceCode": {"name": "floquet_code"}},  # noqa: W505
        # {"qubitParams": {"name": "qubit_maj_ns_e6"}, "surfaceCode": {"name": "floquet_code"}},  # noqa: W505
    ]
)

In [9]:
estimates["physicalCountsFormatted"]

{'runtime': '83 days',
 'rqops': '893.79M',
 'physicalQubits': '31.37M',
 'algorithmicLogicalQubits': '12.51k',
 'algorithmicLogicalDepth': '515.19G',
 'logicalDepth': '515.19G',
 'numTstates': '515.19G',
 'numTfactories': '13',
 'numTfactoryRuns': '39.63G',
 'physicalQubitsForAlgorithm': '30.66M',
 'physicalQubitsForTfactories': '716.04k',
 'physicalQubitsForTfactoriesPercentage': '2.28 %',
 'requiredLogicalQubitErrorRate': '7.76e-20',
 'requiredLogicalTstateErrorRate': '9.71e-16',
 'physicalQubitsPerLogicalQubit': '2.45k',
 'logicalCycleTime': '14 microsecs',
 'clockFrequency': '71.43k',
 'logicalErrorRate': '3.00e-20',
 'tfactoryPhysicalQubits': '55.08k',
 'tfactoryRuntime': '174 microsecs',
 'numInputTstates': '255',
 'numUnitsPerRound': '17, 1',
 'unitNamePerRound': '15-to-1 space efficient, 15-to-1 RM prep',
 'codeDistancePerRound': '9, 29',
 'physicalQubitsPerRound': '55.08k, 52.14k',
 'tfactoryRuntimePerRound': '47 microsecs, 128 microsecs',
 'tstateLogicalErrorRate': '5.68e-16

The QTT will most likely interface with the unformatted output from the Azure QRE:

In [10]:
estimates["physicalCounts"]

{'physicalQubits': 31372890,
 'runtime': 7212638404608000,
 'rqops': 893785715,
 'breakdown': {'algorithmicLogicalQubits': 12513,
  'algorithmicLogicalDepth': 515188457472,
  'logicalDepth': 515188457472,
  'numTstates': 515188457472,
  'clockFrequency': 71428.57142857143,
  'numTfactories': 13,
  'numTfactoryRuns': 39629881344,
  'physicalQubitsForTfactories': 716040,
  'physicalQubitsForAlgorithm': 30656850,
  'requiredLogicalQubitErrorRate': 7.756082776995036e-20,
  'requiredLogicalTstateErrorRate': 9.705186378853888e-16,
  'numTsPerRotation': None,
  'cliffordErrorRate': 0.001}}